In [2]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import seaborn as sns
import pickle

Linear Discriminant Analysis (LDA):
* Supervised method that tries to find the directions of maximum separation between the classes in the data.
* It assumes that the data within each class follows a multivariate normal distribution with a common covariance matrix.
* It projects the data onto a new set of axes, called linear discriminants, that maximize the ratio of the between-class variance (maximizes) to the within-class variance (minimizes). 
* The first linear discriminant is the one that separates the classes the most, the second one is the one that separates the classes the most after removing the effect of the first one, and so on.

Quadratic Discriminant Analysis (QDA) was also tested which does not have the equal covariance between classes assumption like LDA does. 

Step-by-step examples: 
* https://ethanwicker.com/2021-02-07-linear-discriminant-analysis-002/
* https://towardsdatascience.com/linear-discriminant-analysis-in-python-76b8b17817c2
* http://www.sthda.com/english/articles/36-classification-methods-essentials/146-discriminant-analysis-essentials-in-r/

# TODO: transform data to normal using log, root, or Box Cox

# Import data

In [3]:
with open('./../data/subset_master_dataset_w_weather.pkl', 'rb') as f:
        data_ls = pickle.load(f)
print(len(data_ls))

5565


In [4]:
# convert list of dicts to dataframe
data_df = pd.DataFrame.from_dict(data_ls)
print(data_df.shape)
display(data_df.head())
display(data_df.info())
display(data_df.describe())

(5565, 21)


,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,sex,age_at_death,est_stature_inches,...,true_SOD,true_SOD_BP,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADH_thres0,ADH_thres5,ADH_thres10,ADH_thres15
0,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3...,00400124.07.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,...,3.0,torso,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35
1,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3...,00400124.10.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,...,3.0,legs,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35
2,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3...,00400124.11.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,...,3.0,legs,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35
3,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3...,00400124.12.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,...,3.0,head,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35
4,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3...,00400124.13.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.0,...,3.0,head,73.881146,65.115868,32.350972,0.0,1881.8375,1634.070833,1081.443333,362.35


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   new_id              5565 non-null   object        
 1   old_id              5565 non-null   object        
 2   img_path            5565 non-null   object        
 3   img                 5565 non-null   object        
 4   correct_img_date    5565 non-null   datetime64[ns]
 5   date_placed_ARF     5565 non-null   datetime64[ns]
 6   PMI_days            5565 non-null   float64       
 7   sex                 5565 non-null   float64       
 8   age_at_death        5565 non-null   float64       
 9   est_stature_inches  5565 non-null   float64       
 10  est_weight          5565 non-null   float64       
 11  true_SOD            5565 non-null   float64       
 12  true_SOD_BP         5565 non-null   object        
 13  ADD_thres0          5565 non-null   float64     

None

,PMI_days,sex,age_at_death,est_stature_inches,est_weight,true_SOD,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADH_thres0,ADH_thres5,ADH_thres10,ADH_thres15
count,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000,5565.000000
mean,49.664331,0.484816,66.056604,155.004972,184.267565,3.527763,708.290345,690.390777,635.252086,526.452252,17123.411344,16750.067399,15539.655443,13146.278034
std,55.519650,0.499814,14.540983,38.573655,58.632396,1.157832,897.411421,889.248872,863.837315,788.885445,21600.033467,21403.362741,20699.314494,19118.359568
min,0.000000,0.000000,21.000000,0.000000,95.000000,1.000000,0.735417,0.000000,0.000000,0.000000,55.500000,28.450000,0.000000,0.000000
25%,14.000000,0.000000,58.000000,160.020000,144.000000,3.000000,169.334067,153.600799,115.196181,53.183333,4356.442619,3890.429167,3155.655000,1854.531667
50%,31.000000,0.000000,67.000000,167.640000,174.000000,3.000000,409.313144,386.366424,342.317959,228.923148,9914.536190,9472.987500,8609.778333,6151.056667
75%,71.000000,1.000000,76.000000,175.260000,210.000000,4.000000,902.689834,876.523317,834.103166,622.933814,21615.516667,21425.544008,19793.244008,15544.623333
max,450.000000,1.000000,96.000000,195.580000,516.000000,6.000000,5807.064398,5619.145350,5229.522822,4511.635084,140976.538945,137072.899421,126508.944401,110144.846306


# Extract head images

In [5]:
heads_df = data_df[data_df.true_SOD_BP == 'head']
print(heads_df.shape)
display(heads_df.info())

(1834, 21)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1834 entries, 3 to 5563
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   new_id              1834 non-null   object        
 1   old_id              1834 non-null   object        
 2   img_path            1834 non-null   object        
 3   img                 1834 non-null   object        
 4   correct_img_date    1834 non-null   datetime64[ns]
 5   date_placed_ARF     1834 non-null   datetime64[ns]
 6   PMI_days            1834 non-null   float64       
 7   sex                 1834 non-null   float64       
 8   age_at_death        1834 non-null   float64       
 9   est_stature_inches  1834 non-null   float64       
 10  est_weight          1834 non-null   float64       
 11  true_SOD            1834 non-null   float64       
 12  true_SOD_BP         1834 non-null   object        
 13  ADD_thres0          1834 non-null   f

None

# LDA

In [9]:
def qda(df, features_ls, max_pmi=0,):
    if max_pmi != 0:
        df = df[df.PMI_days <= max_pmi].copy()
    # create X and y
    X = df.loc[:,features_ls].copy()
    y = df['true_SOD'].copy()
    
    # standardize X (mean 0 and std 1)
    scaler = StandardScaler().fit(X)  # define standard scaler
    X_stand = scaler.transform(X)  # transform data
    
    # perform lda
    qda = QuadraticDiscriminantAnalysis()
    model = qda.fit(X_stand, y)
    
    # accuracy
    print('# of imgs:',  X.shape[0], ',' , 'features:', features_ls , ',', 'accuracy:', round(qda.score(X_stand, y), 3),)
    

In [5]:
def lda(df, features_ls, max_pmi=0,):
    if max_pmi != 0:
        df = df[df.PMI_days <= max_pmi].copy()
    # create X and y
    X = df.loc[:,features_ls].copy()
    y = df['true_SOD'].copy()
    
    # standardize X (mean 0 and std 1)
    scaler = StandardScaler().fit(X)  # define standard scaler
    X_stand = scaler.transform(X)  # transform data
    
    # perform lda
    lda = LinearDiscriminantAnalysis()
    model = lda.fit(X_stand, y)
    
    # accuracy
    print('# of imgs:',  X.shape[0], ',' , 'features:', features_ls , ',', 'accuracy:', round(lda.score(X_stand, y), 3),)
    

In [10]:
# when no max_pmi
qda(heads_df, ['PMI_days'])
print()
qda(heads_df, ['ADD_thres0'])
qda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
qda(heads_df, ['ADD_thres5'])
qda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
qda(heads_df, ['ADD_thres10'])
qda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
qda(heads_df, ['ADD_thres15'])
qda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
qda(heads_df, ['ADH_thres0'])
qda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
qda(heads_df, ['ADH_thres5'])
qda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
qda(heads_df, ['ADH_thres10'])
qda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
qda(heads_df, ['ADH_thres15'])
qda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'])
qda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])

# of imgs: 1834 , features: ['PMI_days'] , accuracy: 0.468

# of imgs: 1834 , features: ['ADD_thres0'] , accuracy: 0.469
# of imgs: 1834 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.495
# of imgs: 1834 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.495
# of imgs: 1834 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.529

# of imgs: 1834 , features: ['ADD_thres5'] , accuracy: 0.473
# of imgs: 1834 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.488
# of imgs: 1834 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.502
# of imgs: 1834 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.523

# of imgs: 1834 , features: ['ADD_thres10'] , accuracy: 0.477
# of imgs: 1834 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.493
# of imgs: 183

In [182]:
# when no max_pmi
lda(heads_df, ['PMI_days'])
print()
lda(heads_df, ['ADD_thres0'])
lda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
lda(heads_df, ['ADD_thres5'])
lda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
lda(heads_df, ['ADD_thres10'])
lda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
lda(heads_df, ['ADD_thres15'])
lda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
lda(heads_df, ['ADH_thres0'])
lda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
lda(heads_df, ['ADH_thres5'])
lda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
lda(heads_df, ['ADH_thres10'])
lda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])
print()
lda(heads_df, ['ADH_thres15'])
lda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'])
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'])

# of imgs: 1834 , features: ['PMI_days'] , accuracy: 0.478

# of imgs: 1834 , features: ['ADD_thres0'] , accuracy: 0.484
# of imgs: 1834 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.496
# of imgs: 1834 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.503
# of imgs: 1834 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.506

# of imgs: 1834 , features: ['ADD_thres5'] , accuracy: 0.484
# of imgs: 1834 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.499
# of imgs: 1834 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.504
# of imgs: 1834 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.504

# of imgs: 1834 , features: ['ADD_thres10'] , accuracy: 0.487
# of imgs: 1834 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.496
# of imgs: 183

In [11]:
# when max_pmi = 183 (6 months)
qda(heads_df, ['PMI_days'], 183)
print()
qda(heads_df, ['ADD_thres0'], 183)
qda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
qda(heads_df, ['ADD_thres5'], 183)
qda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
qda(heads_df, ['ADD_thres10'], 183)
qda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
qda(heads_df, ['ADD_thres15'], 183)
qda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
qda(heads_df, ['ADH_thres0'], 183)
qda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
qda(heads_df, ['ADH_thres5'], 183)
qda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
qda(heads_df, ['ADH_thres10'], 183)
qda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
qda(heads_df, ['ADH_thres15'], 183)
qda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'], 183)
qda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)

# of imgs: 1811 , features: ['PMI_days'] , accuracy: 0.42

# of imgs: 1811 , features: ['ADD_thres0'] , accuracy: 0.509
# of imgs: 1811 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.49
# of imgs: 1811 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.509
# of imgs: 1811 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.534

# of imgs: 1811 , features: ['ADD_thres5'] , accuracy: 0.522
# of imgs: 1811 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.512
# of imgs: 1811 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.526
# of imgs: 1811 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.562

# of imgs: 1811 , features: ['ADD_thres10'] , accuracy: 0.522
# of imgs: 1811 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.539
# of imgs: 1811 

In [117]:
# when max_pmi = 183 (6 months)
lda(heads_df, ['PMI_days'], 183)
print()
lda(heads_df, ['ADD_thres0'], 183)
lda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
lda(heads_df, ['ADD_thres5'], 183)
lda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
lda(heads_df, ['ADD_thres10'], 183)
lda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
lda(heads_df, ['ADD_thres15'], 183)
lda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
lda(heads_df, ['ADH_thres0'], 183)
lda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
lda(heads_df, ['ADH_thres5'], 183)
lda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
lda(heads_df, ['ADH_thres10'], 183)
lda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)
print()
lda(heads_df, ['ADH_thres15'], 183)
lda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'], 183)
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 183)

# of imgs: 1811 , features: ['PMI_days'] , accuracy: 0.467

# of imgs: 1811 , features: ['ADD_thres0'] , accuracy: 0.487
# of imgs: 1811 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.493
# of imgs: 1811 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.504
# of imgs: 1811 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.506

# of imgs: 1811 , features: ['ADD_thres5'] , accuracy: 0.489
# of imgs: 1811 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.49
# of imgs: 1811 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.501
# of imgs: 1811 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.507

# of imgs: 1811 , features: ['ADD_thres10'] , accuracy: 0.487
# of imgs: 1811 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.499
# of imgs: 1811

In [12]:
# when max_pmi = 91 (3 months)
qda(heads_df, ['PMI_days'], 91)
print()
qda(heads_df, ['ADD_thres0'], 91)
qda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
qda(heads_df, ['ADD_thres5'], 91)
qda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
qda(heads_df, ['ADD_thres10'], 91)
qda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
qda(heads_df, ['ADD_thres15'], 91)
qda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
qda(heads_df, ['ADH_thres0'], 91)
qda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
qda(heads_df, ['ADH_thres5'], 91)
qda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
qda(heads_df, ['ADH_thres10'], 91)
qda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
qda(heads_df, ['ADH_thres15'], 91)
qda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'], 91)
qda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)

# of imgs: 1600 , features: ['PMI_days'] , accuracy: 0.483

# of imgs: 1600 , features: ['ADD_thres0'] , accuracy: 0.531
# of imgs: 1600 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.524
# of imgs: 1600 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.547
# of imgs: 1600 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.572

# of imgs: 1600 , features: ['ADD_thres5'] , accuracy: 0.546
# of imgs: 1600 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.544
# of imgs: 1600 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.556
# of imgs: 1600 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.588

# of imgs: 1600 , features: ['ADD_thres10'] , accuracy: 0.533
# of imgs: 1600 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.544
# of imgs: 160

In [118]:
# when max_pmi = 91 (3 months)
lda(heads_df, ['PMI_days'], 91)
print()
lda(heads_df, ['ADD_thres0'], 91)
lda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
lda(heads_df, ['ADD_thres5'], 91)
lda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
lda(heads_df, ['ADD_thres10'], 91)
lda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
lda(heads_df, ['ADD_thres15'], 91)
lda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
lda(heads_df, ['ADH_thres0'], 91)
lda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
lda(heads_df, ['ADH_thres5'], 91)
lda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
lda(heads_df, ['ADH_thres10'], 91)
lda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)
print()
lda(heads_df, ['ADH_thres15'], 91)
lda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'], 91)
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 91)

# of imgs: 1600 , features: ['PMI_days'] , accuracy: 0.483

# of imgs: 1600 , features: ['ADD_thres0'] , accuracy: 0.49
# of imgs: 1600 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.499
# of imgs: 1600 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.506
# of imgs: 1600 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.52

# of imgs: 1600 , features: ['ADD_thres5'] , accuracy: 0.492
# of imgs: 1600 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.501
# of imgs: 1600 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.506
# of imgs: 1600 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.519

# of imgs: 1600 , features: ['ADD_thres10'] , accuracy: 0.497
# of imgs: 1600 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.506
# of imgs: 1600 

In [13]:
# when max_pmi = 31 (1 month)
qda(heads_df, ['PMI_days'], 31)
print()
qda(heads_df, ['ADD_thres0'], 31)
qda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
qda(heads_df, ['ADD_thres5'], 31)
qda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
qda(heads_df, ['ADD_thres10'], 31)
qda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
qda(heads_df, ['ADD_thres15'], 31)
qda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
qda(heads_df, ['ADH_thres0'], 31)
qda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
qda(heads_df, ['ADH_thres5'], 31)
qda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
qda(heads_df, ['ADH_thres10'], 31)
qda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
qda(heads_df, ['ADH_thres15'], 31)
qda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'], 31)
qda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)

# of imgs: 972 , features: ['PMI_days'] , accuracy: 0.487

# of imgs: 972 , features: ['ADD_thres0'] , accuracy: 0.543
# of imgs: 972 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.561
# of imgs: 972 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.595
# of imgs: 972 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.6

# of imgs: 972 , features: ['ADD_thres5'] , accuracy: 0.547
# of imgs: 972 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.581
# of imgs: 972 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.604
# of imgs: 972 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.608

# of imgs: 972 , features: ['ADD_thres10'] , accuracy: 0.561
# of imgs: 972 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.592
# of imgs: 972 , features: 

In [119]:
# when max_pmi = 31 (1 month)
lda(heads_df, ['PMI_days'], 31)
print()
lda(heads_df, ['ADD_thres0'], 31)
lda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
lda(heads_df, ['ADD_thres5'], 31)
lda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
lda(heads_df, ['ADD_thres10'], 31)
lda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
lda(heads_df, ['ADD_thres15'], 31)
lda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
lda(heads_df, ['ADH_thres0'], 31)
lda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
lda(heads_df, ['ADH_thres5'], 31)
lda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
lda(heads_df, ['ADH_thres10'], 31)
lda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)
print()
lda(heads_df, ['ADH_thres15'], 31)
lda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'], 31)
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 31)

# of imgs: 972 , features: ['PMI_days'] , accuracy: 0.485

# of imgs: 972 , features: ['ADD_thres0'] , accuracy: 0.552
# of imgs: 972 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.604
# of imgs: 972 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.588
# of imgs: 972 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.584

# of imgs: 972 , features: ['ADD_thres5'] , accuracy: 0.581
# of imgs: 972 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.59
# of imgs: 972 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.595
# of imgs: 972 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.584

# of imgs: 972 , features: ['ADD_thres10'] , accuracy: 0.573
# of imgs: 972 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.596
# of imgs: 972 , features:

In [143]:
# when max_pmi = 14 (2 weeks)
lda(heads_df, ['PMI_days'], 14)
print()
lda(heads_df, ['ADD_thres0'], 14)
lda(heads_df, ['ADD_thres0', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 14)
print()
lda(heads_df, ['ADD_thres5'], 14)
lda(heads_df, ['ADD_thres5', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 14)
print()
lda(heads_df, ['ADD_thres10'], 14)
lda(heads_df, ['ADD_thres10', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADD_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 14)
print()
lda(heads_df, ['ADD_thres15'], 14)
lda(heads_df, ['ADD_thres15', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADD_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 14)
print()
lda(heads_df, ['ADH_thres0'], 14)
lda(heads_df, ['ADH_thres0', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADH_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 14)
print()
lda(heads_df, ['ADH_thres5'], 14)
lda(heads_df, ['ADH_thres5', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADH_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 14)
print()
lda(heads_df, ['ADH_thres10'], 14)
lda(heads_df, ['ADH_thres10', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADH_thres10', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 14)
print()
lda(heads_df, ['ADH_thres15'], 14)
lda(heads_df, ['ADH_thres15', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches'], 14)
lda(heads_df, ['ADH_thres15', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'], 14)

# of imgs: 513 , features: ['PMI_days'] , accuracy: 0.47

# of imgs: 513 , features: ['ADD_thres0'] , accuracy: 0.6
# of imgs: 513 , features: ['ADD_thres0', 'est_weight', 'est_stature_inches'] , accuracy: 0.583
# of imgs: 513 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.63
# of imgs: 513 , features: ['ADD_thres0', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.641

# of imgs: 513 , features: ['ADD_thres5'] , accuracy: 0.6
# of imgs: 513 , features: ['ADD_thres5', 'est_weight', 'est_stature_inches'] , accuracy: 0.606
# of imgs: 513 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches'] , accuracy: 0.608
# of imgs: 513 , features: ['ADD_thres5', 'sex', 'est_weight', 'est_stature_inches', 'age_at_death'] , accuracy: 0.637

# of imgs: 513 , features: ['ADD_thres10'] , accuracy: 0.602
# of imgs: 513 , features: ['ADD_thres10', 'est_weight', 'est_stature_inches'] , accuracy: 0.593
# of imgs: 513 , features: ['AD